# Guitar Chord Prediction
---

Dependencies

In [41]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import seaborn as sns
import random
import os

Seed for reproducability

In [42]:
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)
os.environ["PYTHONHASHSEED"] = str(SEED)

NdLinear

In [2]:
!pip install ndlinear

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
from ndlinear import NdLinear

### Data Loading and Preprocessing

In [4]:
df = pd.read_csv("chord-fingers.csv", sep=";")
df.head()

,CHORD_ROOT,CHORD_TYPE,CHORD_STRUCTURE,FINGER_POSITIONS,NOTE_NAMES
0,A#,13,1;3;5;b7;9;11;13,"x,1,0,2,3,4","A#,C##,G#,B#,F##"
1,A#,13,1;3;5;b7;9;11;13,"4,x,3,2,1,1","A#,G#,B#,C##,F##"
2,A#,13,1;3;5;b7;9;11;13,"1,x,1,2,3,4","A#,G#,C##,F##,B#"
3,A#,7(#9),1;3;5;b7;#9,"x,1,0,2,4,3","A#,C##,G#,B##,E#"
4,A#,7(#9),1;3;5;b7;#9,"2,1,3,3,3,x","A#,C##,G#,B##,E#"


Using Label Encoder for each feature.

In [6]:
rootEncoder = LabelEncoder()
xRoot = rootEncoder.fit_transform(df["CHORD_ROOT"])

structureEncoder = LabelEncoder()
xStruct = structureEncoder.fit_transform(df["CHORD_STRUCTURE"])
xFingers = (
    df["FINGER_POSITIONS"]
    .str.split(",", expand=True)
    .replace("x", -1)
    .astype(float)
    .values
)

targetEncoder = LabelEncoder()

X = np.hstack([xRoot.reshape(-1,1), xStruct.reshape(-1,1), xFingers])
y = targetEncoder.fit_transform(df["CHORD_TYPE"])

Split into train/test, conver to torch tensors

In [43]:
# Seed
g = torch.Generator().manual_seed(SEED)

# Split
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to Torch tensors
xTrainTensor = torch.tensor(xTrain, dtype=torch.float32)
yTrainTensor = torch.tensor(yTrain, dtype=torch.long)
xTestTensor = torch.tensor(xTest, dtype=torch.float32)
yTestTensor = torch.tensor(yTest, dtype=torch.long)

# Data Loader
trainDataset = DataLoader(TensorDataset(xTrainTensor, yTrainTensor), batch_size=64, shuffle=True, generator=g)
testDataset = DataLoader(TensorDataset(xTestTensor, yTestTensor), batch_size=64, generator=g)

### Define the Multilayer Perceptron Models

Default Model, using nn.Linear

In [9]:
class MLP_Linear(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )

    def forward(self, x):
        return self.model(x)

Modified model, with NdLinear

In [17]:
class MLP_NdLinear(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            NdLinear((input_dim,), (64,)),
            nn.ReLU(),
            NdLinear((64,), (128,)),
            nn.ReLU(),
            NdLinear((128,), (output_dim,))
        )

    def forward(self, x):
        return self.model(x)

Training

In [11]:
def train(model, train_loader, criterion, optimizer):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

Evaluation

In [31]:
def evaluate(model, test_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            output = model(X_batch)
            preds = torch.argmax(output, dim=1)
            all_preds.extend(preds.tolist())
            all_labels.extend(y_batch.tolist())
    return accuracy_score(all_labels, all_preds), classification_report(all_labels, all_preds, zero_division=0)

Run the training and evaluation on both models.

In [50]:
input_dim = xTrain.shape[1]
output_dim = len(np.unique(y))

# Linear baseline
model_linear = MLP_Linear(input_dim, output_dim)
optimizer_linear = torch.optim.Adam(model_linear.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# NdLinear model
model_ndlinear = MLP_NdLinear(input_dim, output_dim)
optimizer_nd = torch.optim.Adam(model_ndlinear.parameters(), lr=0.01)

# Train
for epoch in range(75):
    train(model_linear, trainDataset, criterion, optimizer_linear)
    train(model_ndlinear, trainDataset, criterion, optimizer_nd)

# Evaluate
acc1, report1 = evaluate(model_linear, testDataset)
acc2, report2 = evaluate(model_ndlinear, testDataset)
print("Accuracy (Default nn.Linear):\n", acc1)
print("Accuracy (NdLinear):\n", acc2)

Accuracy (Default nn.Linear):
 0.8045540796963947
Accuracy (NdLinear):
 0.8311195445920304
